In [1]:
import os
import openai
import json

import sys
# setting path
sys.path.append('../')

import itertools

from descriptor_strings import stringtolist

openai.api_key = "" #FILL IN YOUR OWN HERE

In [2]:
def generate_prompt(category_name: str):
    # you can replace the examples with whatever you want; these were random and worked, could be improved
    return f"""Q: What are useful visual features for distinguishing a Persian Cat in a photo? These features should be visually distinct and have limited overlap with each other. Please list 6 of them and sort them in descending order of importance. For each item, you should be concise and precise, and use no more than five words. No ambiguous answers.
A: There are several useful visual features to tell there is a Persian Cat in a photo:
- Flat, round face
- Long, dense fur
- Short, stubby nose
- Large, round eyes
- Short, thick legs
- Flowing tail

Q: What are useful visual features for distinguishing a Tench in a photo? These features should be visually distinct and have limited overlap with each other. Please list 6 of them and sort them in descending order of importance. For each item, you should be concise and precise, and use no more than five words. No ambiguous answers.
A: There are several useful visual features to tell there is a Tench in a photo:
- Olive-green body color
- Thick, slimy skin
- Small, red eyes
- Rounded fins
- Cylindrical body shape
- Barbels near mouth

Q: What are useful visual features for distinguishing a {category_name} in a photo? These features should be visually distinct and have limited overlap with each other. Please list 6 of them and sort them in descending order of importance. For each item, you should be concise and precise, and use no more than five words. No ambiguous answers.
A: There are several useful visual features to tell there is a {category_name} in a photo:
"""

In [3]:
# generator 
def partition(lst, size):
    for i in range(0, len(lst), size):
        yield list(itertools.islice(lst, i, i + size))

In [4]:
def obtain_descriptors_and_save(filename, class_list):
    responses = {}
    descriptors = {}
    
    prompts = [generate_prompt(category.replace('_', ' ')) for category in class_list]
    
    
    # most efficient way is to partition all prompts into the max size that can be concurrently queried from the OpenAI API
    responses = [openai.Completion.create(model="text-davinci-003",
                                            prompt=prompt_partition,
                                            temperature=0.,
                                            max_tokens=100,
                                            ) for prompt_partition in partition(prompts, 20)]
    response_texts = [r["text"] for resp in responses for r in resp['choices']]
    descriptors_list = [stringtolist(response_text) for response_text in response_texts]
    descriptors = {cat: descr for cat, descr in zip(class_list, descriptors_list)}

    # save descriptors to json file
    if not filename.endswith('.json'):
        filename += '.json'
    with open(filename, 'w') as fp:
        json.dump(descriptors, fp)

In [5]:
with open('/usa/tangli/Projects/concept_exp/descriptors/my_imagenet.json', 'r') as file:
    data = json.load(file)
keys = list(data.keys())

In [8]:
obtain_descriptors_and_save('/usa/tangli/Projects/concept_exp/descriptors/my_cub.json', keys)